In [1]:
#r "./bin/Debug/net5.0/PreStormCore.dll"
#r "./bin/Debug/net5.0/Demo.dll"
using System.Diagnostics;
using PreStormCore;

var crimes = new Sfcrimes.Service().CrimeLocations.Download(extraParameters: "outSR=4326", keepQuerying: true, degreeOfParallelism: 8).ToList();
var cities = new Usa.Service().Cities.Download(keepQuerying: true, degreeOfParallelism: 4).ToList();
var highways = new Usa.Service().Highways.Download().ToList();
var states = new Usa.Service().States.Download().ToList();

public static void Show<T>(this IEnumerable<T> fatures) where T: Feature {
    fatures.ToKml().Save("doc.kml");
    Process.Start("explorer.exe", "doc.kml");
}


In [1]:
crimes.Take(4)


index,Category,Descript,Pddistrict,Datetime,Geometry,OID,IsDataBound,IsDirty
0,DUI/DRUNKENNESS,UNDER INFLUENCE OF ALCOHOL IN A PUBLIC PLACE,MISSION,2012-03-10 03:20:00Z,PreStormCore.Point,1,True,False
1,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM OF VEHICLES",BAYVIEW,2012-03-10 05:16:00Z,PreStormCore.Point,2,True,False
2,OTHER OFFENSES,LOST/STOLEN LICENSE PLATE,MISSION,2012-03-09 21:00:00Z,PreStormCore.Point,3,True,False
3,MISSING PERSON/RUNAWAY,FOUND PERSON,RICHMOND,2012-03-10 12:00:00Z,PreStormCore.Point,4,True,False


In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Bar(crimes.GroupBy(x => x.Pddistrict).Select(x => Tuple.Create(x.Key, x.Count())))


In [1]:
#r "nuget: XPlot.Plotly, 3.0.1"
XPlot.Plotly.Chart.Pie(crimes.GroupBy(x => x.Category).OrderByDescending(x => x.Count()).Select(x => Tuple.Create(x.Key, x.Count())))


In [1]:
crimes.Take(4).Select(x => x.ToText())


index,value
0,"""1"",""DUI/DRUNKENNESS"",""UNDER INFLUENCE OF ALCOHOL IN A PUBLIC PLACE"",""MISSION"",""2012-03-10T03:20:00.0000000Z"""
1,"""2"",""VANDALISM"",""MALICIOUS MISCHIEF, VANDALISM OF VEHICLES"",""BAYVIEW"",""2012-03-10T05:16:00.0000000Z"""
2,"""3"",""OTHER OFFENSES"",""LOST/STOLEN LICENSE PLATE"",""MISSION"",""2012-03-09T21:00:00.0000000Z"""
3,"""4"",""MISSING PERSON/RUNAWAY"",""FOUND PERSON"",""RICHMOND"",""2012-03-10T12:00:00.0000000Z"""


In [1]:
cities.Where(x => x.St == "OH").Show()


In [1]:
highways.Show()


In [1]:
states.Where(x => x.StateAbbr == "AK").Show()
